# Homologation of **colonias**

The purpose of this notebook is to homologate the *colonia* values in the *crimes_clean* dataset using a JSON file called *catalogo-de-colonias.json*, which includes names and keys of the location in detail.

In [8]:
import pandas as pd
import zipfile
import numpy as np
import matplotlib.pyplot as plt
import json

#Path
#Insert your local repo path to file 
repo_path = "INSERT YOUR LOCAL SDC-SECURITY REPO PATH HERE"
repo_path ='D:\Archivos\Social Data Challenge\sdc-security'

In [3]:
crime_data_path = "\datasets\crimes_clean.csv.zip"

zf = zipfile.ZipFile(repo_path+crime_data_path) 
crimes = pd.read_csv(zf.open('crimes_clean.csv'))
print(crimes.shape)
crimes.head()


(781226, 23)


,idcarpeta,delito,categoria,alcaldia,colonia,sexo,edad,tipopersona,calidadjuridica,anio_denuncia,...,competencia,anio_hecho,mes_hecho,fecha_hecho,hora_hecho,colonia_alt,crimen_lat,crimen_lon,hecho_time,denuncia_time
0,8324429,FRAUDE,DELITO DE BAJO IMPACTO,ALVARO OBREGON,GUADALUPE INN,Masculino,62,FISICA,OFENDIDO,2019,...,FUERO COMUN,2018,8,2018-08-29,12:00:00,GUADALUPE INN,19.36125,-99.18314,2018-08-29 12:00:00,2019-01-04 12:19:00
1,8324430,"PRODUCCIÓN, IMPRESIÓN, ENAJENACIÓN, DISTRIBUCI...",DELITO DE BAJO IMPACTO,AZCAPOTZALCO,VICTORIA DE LAS DEMOCRACIAS,Femenino,38,FISICA,VICTIMA Y DENUNCIANTE,2019,...,FUERO COMUN,2018,12,2018-12-15,15:00:00,VICTORIA DE LAS DEMOCRACIAS,19.47181,-99.16458,2018-12-15 15:00:00,2019-01-04 12:20:00
2,8324431,ROBO A TRANSEUNTE SALIENDO DEL BANCO CON VIOLE...,ROBO A CUENTAHABIENTE SALIENDO DEL CAJERO CON ...,COYOACAN,COPILCO UNIVERSIDAD ISSSTE,Masculino,42,FISICA,VICTIMA Y DENUNCIANTE,2019,...,FUERO COMUN,2018,12,2018-12-22,15:30:00,COPILCO EL BAJO,19.33797,-99.18611,2018-12-22 15:30:00,2019-01-04 12:23:00
3,8324435,ROBO DE VEHICULO DE SERVICIO PARTICULAR SIN VI...,ROBO DE VEHÍCULO CON Y SIN VIOLENCIA,IZTACALCO,AGRÍCOLA PANTITLAN,Masculino,35,FISICA,VICTIMA Y DENUNCIANTE,2019,...,FUERO COMUN,2019,1,2019-01-04,06:00:00,PANTITLAN V,19.40327,-99.05983,2019-01-04 06:00:00,2019-01-04 12:27:00
4,8324438,ROBO DE MOTOCICLETA SIN VIOLENCIA,ROBO DE VEHÍCULO CON Y SIN VIOLENCIA,IZTAPALAPA,PROGRESISTA,Masculino,39,FISICA,VICTIMA,2019,...,FUERO COMUN,2019,1,2019-01-03,20:00:00,LAS AMERICAS (U HAB),19.35480,-99.06324,2019-01-03 20:00:00,2019-01-04 12:35:00


In [21]:
catalog_path = "\datasets\catalogo-de-colonias.json"

#catalog = pd.read_json(repo_path + catalog_path)

# Load the JSON file
with open(repo_path + catalog_path, 'r', encoding='utf-8') as json_file:
    data = json.load(json_file)

# Extracting features from the JSON data
features = data['features']

# Initialize lists to store column values
polygon_list = []
properties_list = []

# Extracting values from features
for feature in features:
    polygon_list.append(feature['geometry']['coordinates'][0])  # Extracting the first set of coordinates
    properties_list.append(feature['properties'])

# Creating the DataFrame
catalog = pd.DataFrame(properties_list)
catalog['polygon'] = polygon_list

# Reordering columns
column_order = ['cve_col', 'clasif', 'colonia', 'alc', 'cve_alc', 'cve_ent', 'entidad', 'polygon']
catalog= catalog[column_order]

# Display the DataFrame
catalog.head()


,cve_col,clasif,colonia,alc,cve_alc,cve_ent,entidad,polygon
0,002-001,Colonia,Aguilera,Azcapotzalco,002,09,Ciudad de México,"[[-99.15913100449802, 19.47261175522339], [-99..."
1,002-002,Colonia,Aldana,Azcapotzalco,002,09,Ciudad de México,"[[-99.15171083760227, 19.468190578205796], [-9..."
2,002-003,Colonia,Ampliacion Cosmopolita,Azcapotzalco,002,09,Ciudad de México,"[[-99.16361818076089, 19.472229875914007], [-9..."
3,002-004,Colonia,Ampliacion Del Gas,Azcapotzalco,002,09,Ciudad de México,"[[-99.16151817603475, 19.46774486135027], [-99..."
4,002-005,Colonia,Ampliacion Petrolera,Azcapotzalco,002,09,Ciudad de México,"[[-99.19761990655803, 19.482307007833892], [-9..."


In [22]:
#Reviewing how does a value inside 'polygon looks like'
catalog["polygon"].loc[0]

[[-99.15913100449802, 19.47261175522339],
 [-99.15905951558406, 19.473285887676024],
 [-99.15900881461089, 19.47368593671133],
 [-99.15794643848542, 19.473632739085684],
 [-99.15616814209562, 19.47354369195322],
 [-99.15442193075381, 19.473380151110387],
 [-99.15266341191717, 19.47321472340822],
 [-99.15275750249315, 19.47264915251562],
 [-99.1528071338663, 19.4723511157457],
 [-99.15292459787696, 19.471676013892726],
 [-99.15351363460762, 19.47176437869369],
 [-99.15501209831677, 19.47198914429913],
 [-99.15500458310345, 19.472030004073765],
 [-99.15510539658004, 19.47200313753843],
 [-99.15520542932478, 19.47197558462651],
 [-99.15519777130277, 19.47201698619364],
 [-99.15580174927318, 19.472107529770657],
 [-99.15640573743234, 19.47219806231646],
 [-99.15814628678987, 19.472462267199276],
 [-99.15913100449802, 19.47261175522339]]

In [24]:
print(catalog.clasif.unique())
print(catalog.entidad.unique())
print(catalog.cve_ent.unique())

['Colonia' 'Pueblos y Barrios Originarios']
['Ciudad de México']
['09']


In [25]:
#erasing cve_ent and entidad, since it is the same on the whole catalog
del catalog["cve_ent"]
del catalog["entidad"]

catalog.head()

,cve_col,clasif,colonia,alc,cve_alc,polygon
0,002-001,Colonia,Aguilera,Azcapotzalco,002,"[[-99.15913100449802, 19.47261175522339], [-99..."
1,002-002,Colonia,Aldana,Azcapotzalco,002,"[[-99.15171083760227, 19.468190578205796], [-9..."
2,002-003,Colonia,Ampliacion Cosmopolita,Azcapotzalco,002,"[[-99.16361818076089, 19.472229875914007], [-9..."
3,002-004,Colonia,Ampliacion Del Gas,Azcapotzalco,002,"[[-99.16151817603475, 19.46774486135027], [-99..."
4,002-005,Colonia,Ampliacion Petrolera,Azcapotzalco,002,"[[-99.19761990655803, 19.482307007833892], [-9..."


After erasing some columns that will not be used, spanish accented character should be replaced by their non-accented counterpart

In [32]:
pip install unidecode


     ---------------------------------------- 0.0/235.9 kB ? eta -:--:--
     - -------------------------------------- 10.2/235.9 kB ? eta -:--:--
     - -------------------------------------- 10.2/235.9 kB ? eta -:--:--
     ---- -------------------------------- 30.7/235.9 kB 325.1 kB/s eta 0:00:01
     --------------------- -------------- 143.4/235.9 kB 774.0 kB/s eta 0:00:01
     -------------------------------------- 235.9/235.9 kB 1.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [33]:
from unidecode import unidecode
columns_to_process = ['clasif', 'colonia', 'alc']

for col in columns_to_process:
    catalog[col] = catalog[col].apply(unidecode)


In [34]:
#Converting text to uppercase
catalog["clasif"] = catalog["clasif"].str.upper()
catalog["colonia"]= catalog["colonia"].str.upper()
catalog["alc"]= catalog["alc"].str.upper()

Focusing on the CUAUHTEMOC district, a slice of this catalog containg only information related to that district will be created


In [35]:
catalog_cuau = catalog.loc[ catalog["alc"] == 'CUAUHTEMOC'].copy()
catalog_cuau.head()

,cve_col,clasif,colonia,alc,cve_alc,polygon
1357,015-001,COLONIA,ALGARIN,CUAUHTEMOC,015,"[[-99.14499269220823, 19.403872867220105], [-9..."
1358,015-002,COLONIA,AMPL. ASTURIAS,CUAUHTEMOC,015,"[[-99.13609631370444, 19.40605476870975], [-99..."
1359,015-003,COLONIA,ASTURIAS,CUAUHTEMOC,015,"[[-99.136483965521, 19.403389149126806], [-99...."
1360,015-004,COLONIA,ATLAMPA,CUAUHTEMOC,015,"[[-99.16375908271087, 19.45579888070804], [-99..."
1361,015-005,COLONIA,BUENAVISTA,CUAUHTEMOC,015,"[[-99.15567247450599, 19.439779410122352], [-9..."


In [37]:
# How many colonias are related to Cuauhtemoc district on this catalog?
catalog_cuau.colonia.unique().shape[0]

33

The next step is to homologate the *colonia* names in the *crimes* dataset and fix the names when needed so that, in future merges with other datasets, the catalog is used as the naming convention.

In [38]:
crimes.columns

Index(['idcarpeta', 'delito', 'categoria', 'alcaldia', 'colonia', 'sexo',
       'edad', 'tipopersona', 'calidadjuridica', 'anio_denuncia',
       'mes_denuncia', 'fecha_denuncia', 'hora_denuncia', 'competencia',
       'anio_hecho', 'mes_hecho', 'fecha_hecho', 'hora_hecho', 'colonia_alt',
       'crimen_lat', 'crimen_lon', 'hecho_time', 'denuncia_time'],
      dtype='object')

In [ ]:
from unidecode import unidecode

columns_to_process = ['delito', 'categoria', 'alcaldia', 'colonia', 'colonia_alt', \
                      'tipopersona','calidadjuridica,competencia']

for col in columns_to_process:
    catalog[col] = catalog[col].apply(unidecode)
    
#Converting text to uppercase
catalog["clasif"] = catalog["clasif"].str.upper()
catalog["colonia"]= catalog["colonia"].str.upper()
catalog["alc"]= catalog["alc"].str.upper()


In [39]:
print(unidecode('CAFÉ'))

CAFE


In [5]:
from fuzzywuzzy import fuzz, process

def get_best_substitute(substitute_this, potential_substitutes):
    # Get the best match and its score using fuzzy matching
    best_match, score = process.extractOne(substitute_this, potential_substitutes)
    if score >= 90:  # You can adjust the threshold as needed
        return best_match
    else:
        return None

D:\Programas\jupyterlab-desktop\jlab_server\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
#Get the unique pairs 'alcalduia'-'colonia'
missing_alcaldia_colonia_pairs = consolidated_nulls.drop_duplicates(subset=['alcaldia', 'colonia'])[['alcaldia', 'colonia']].values.tolist()
original_alcaldia_colonia_pairs = crimes_final.drop_duplicates(subset=['alcaldia', 'colonia'])[['alcaldia', 'colonia']].values.tolist()

#missing_colonias =list(consolidated_nulls.colonia.unique())
#substitute_list = list(crimes_final.colonia.unique())

#missing_colonias.sort()
#substitute_list.sort()
missing_alcaldia_colonia_pairs.sort()
original_alcaldia_colonia_pairs.sort()
print(len(missing_alcaldia_colonia_pairs))

#Getting the alcaldia - colonia pair inside a single string with '-' as separator 
missing_colonias  = [f"{pair[0]} - {pair[1]}" for pair in missing_alcaldia_colonia_pairs]
original_colonias = [f"{pair[0]} - {pair[1]}" for pair in original_alcaldia_colonia_pairs]